# Harnessing Weather Insights for Accurate Energy Load Forecasting

In [53]:
%pip install -r requirements.txt

  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------- ------------------------ 4.5/11.5 MB 29.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 35.8 MB/s eta 0:00:00
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


### Import important libraries

In [16]:
import platform
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pathlib import Path

In [3]:
# Initialize via the full spark path
if platform.system() == 'Windows':
    print("Windows OS detected")
    findspark.init("C:/Spark/spark-3.5.4-bin-hadoop3") # For my local machine
else:
    findspark.init("/usr/local/spark/")

Windows OS detected


In [4]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Linear Regression Model") \
    .config("spark.executor.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.15.0") \
    .config("spark.executor.heartbeatInterval", "100s") \
    .config("spark.sql.session.timeZone", "UTC") \
    .getOrCreate()

   
# Main entry point for Spark functionality. A SparkContext represents the
# connection to a Spark cluster, and can be used to create :class:`RDD` and
# broadcast variables on that cluster.      
sc = spark.sparkContext

### Preprocessing I: Read in Weather and Load Data

In [14]:
# Read in the data

# Folder Structure
# data
# |-- geosphere
# |   |-- YYYY
# |      |-- MM.csv
# |      |-- MM.csv
# |
# |-- transparency
# |   |-- YYYY
# |      |-- MM.xml
# |      |-- MM.xml

# Loop through the geosphere folder and read in the data

# Define the base data folder
base_path = Path("./data/geosphere")

# Collect all data frames first to optimize the union operation
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.csv"):
            print(f"Reading in {month_file}")

            df = spark.read.csv(str(month_file), header=True, inferSchema=True)

            # Convert the time column (string) to a timestamp
            df = df.withColumn("time", to_timestamp(col("time"), "yyyy-MM-dd'T'HH:mmXXX"))
            
            dfs.append(df)
            
            print(f"Read in {df.count()} rows")

if dfs:
    # Combine all DataFrames
    weather = dfs[0]
    for df in dfs[1:]:
        weather = weather.union(df)

    # Aggregate measurements (average from different stations)
    weather = (
        weather.groupBy("time")
        .agg(
            avg("rr").alias("avg_rr"),
            avg("tl").alias("avg_tl"),
            avg("p").alias("avg_p"),
            avg("so_h").alias("avg_so_h"),
            avg("ff").alias("avg_ff"),
        )
        .orderBy("time")
    )

    # Log-transform the measurements (data for rainfall and windspeed is skewed)
    weather = weather.withColumn("log_rr", log1p(col("avg_rr")))
    weather = weather.withColumn("log_ff", log1p(col("avg_ff")))
    
    weather.drop("avg_rr", "avg_ff")
    
    weather.show(10, truncate=False)
    
    print(weather.count())
    weather.printSchema()
else:
    print("No data found")


Reading in data\geosphere\2022\01.csv
Read in 1490 rows
Reading in data\geosphere\2022\02.csv
Read in 1346 rows
Reading in data\geosphere\2022\03.csv
Read in 1490 rows
Reading in data\geosphere\2022\04.csv
Read in 1442 rows
Reading in data\geosphere\2022\05.csv
Read in 1490 rows
Reading in data\geosphere\2022\06.csv
Read in 1442 rows
Reading in data\geosphere\2022\07.csv
Read in 1490 rows
Reading in data\geosphere\2022\08.csv
Read in 1490 rows
Reading in data\geosphere\2022\09.csv
Read in 1442 rows
Reading in data\geosphere\2022\10.csv
Read in 1490 rows
Reading in data\geosphere\2022\11.csv
Read in 1442 rows
Reading in data\geosphere\2022\12.csv
Read in 1490 rows
Reading in data\geosphere\2023\01.csv
Read in 1490 rows
Reading in data\geosphere\2023\02.csv
Read in 1346 rows
Reading in data\geosphere\2023\03.csv
Read in 1490 rows
Reading in data\geosphere\2023\04.csv
Read in 1442 rows
Reading in data\geosphere\2023\05.csv
Read in 1490 rows
Reading in data\geosphere\2023\06.csv
Read in 14

In [6]:
# Loop through the transparency folder and read in the energy data

# Define base path for transparency data
base_path = Path("./data/transparency")

# Collect DataFrames before performing union (optimization)
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.xml"):
            print(f"Reading transparency data: {month_file}")

            # Read XML data
            df = spark.read.format('xml').option("rowTag", "GL_MarketDocument").load(str(month_file))

            # Extract and explode relevant fields
            df_filtered = df.select(
                col("TimeSeries.Period.timeInterval.start").alias("start_time"),
                col("TimeSeries.Period.timeInterval.end").alias("end_time"),
                col("TimeSeries.Period.resolution").alias("resolution"),
                explode(col("TimeSeries.Period.Point")).alias("Point")  # Flatten Points
            ).select(
                col("start_time"),
                col("end_time"),
                col("resolution"),
                col("Point.position").cast("int").alias("position"),
                col("Point.quantity").cast("double").alias("quantity")
            )

            # Convert ISO 8601 duration (e.g., "PT15M") to minutes dynamically
            df_fixed = df_filtered.withColumn(
                "interval_minutes",
                expr("CAST(SUBSTRING(resolution, 3, LENGTH(resolution) - 3) AS INT)")  # Extracts "15" from "PT15M"
            ).withColumn(
                "actual_time",
                expr("start_time + (position - 1) * interval_minutes * interval 1 minute")
            ).select(
                col("actual_time"),
                col("quantity")
            )
            
            # Aggregate to hourly intervals
            df_hourly = df_fixed.withColumn(
                "hourly_time", date_trunc("hour", col("actual_time"))  # Round down to the hour
            ).groupBy("hourly_time").agg(
                sum("quantity").alias("hourly_quantity")  # Sum all sub-hourly measurements
            )

            # Rename to match the structure of other time series
            df_hourly = df_hourly.select(
                col("hourly_time").alias("timestamp"), col("hourly_quantity").alias("quantity")
            ).orderBy("timestamp")
            
            print(f"Read in {df_hourly.count()} values")
            
            # Append DataFrame to list
            dfs.append(df_hourly)

# Merge all collected DataFrames
if dfs:
    Load = dfs[0]
    for df in dfs[1:]:
        Load = Load.union(df)

    Load.show(10)
    Load.printSchema()
else:
    print("No data found.")



Reading transparency data: data\transparency\2022\01.xml
Read in 744 values
Reading transparency data: data\transparency\2022\02.xml
Read in 672 values
Reading transparency data: data\transparency\2022\03.xml
Read in 744 values
Reading transparency data: data\transparency\2022\04.xml
Read in 720 values
Reading transparency data: data\transparency\2022\05.xml
Read in 744 values
Reading transparency data: data\transparency\2022\06.xml
Read in 720 values
Reading transparency data: data\transparency\2022\07.xml
Read in 744 values
Reading transparency data: data\transparency\2022\08.xml
Read in 744 values
Reading transparency data: data\transparency\2022\09.xml
Read in 720 values
Reading transparency data: data\transparency\2022\10.xml
Read in 744 values
Reading transparency data: data\transparency\2022\11.xml
Read in 720 values
Reading transparency data: data\transparency\2022\12.xml
Read in 744 values
Reading transparency data: data\transparency\2023\01.xml
Read in 744 values
Reading tran

### Preprocessing II: Combine both Data Frames

In [57]:
if Load is not None and weather is not None:
    # Join the data into a single DataFrame
    data = Load.join(weather, Load.timestamp == weather.time, "inner").drop("time")
    
    # Rename columns for better understanding
    data = data.withColumnRenamed("timestamp", "time")
    data = data.withColumnRenamed("quantity", "load")

    data = data.withColumnRenamed("log_rr", "rainfall")
    data = data.withColumnRenamed("avg_tl", "temperature")
    data = data.withColumnRenamed("avg_p", "pressure")
    data = data.withColumnRenamed("avg_so_h", "sunshine_duration")
    data = data.withColumnRenamed("log_ff", "wind_speed")
    
    # Add Lag features
    
    # Define window for lagging (ordered by time)
    window_spec = Window().partitionBy().orderBy("time")

    # Add lag features (previous hour's values)
    data = data.withColumn("rainfall_lag_1", lag("rainfall", 1).over(window_spec))
    data = data.withColumn("wind_speed_lag_1", lag("wind_speed", 1).over(window_spec))
    
    # Add lag features (previous day's values)
    data = data.withColumn("load_lag_24", lag("load", 24).over(window_spec))
    
    # Drop rows with missing values
    data = data.dropna()
    
    # Reorder the columns
    data = data.select("time",
                        "load", 
                        "load_lag_24",
                        "rainfall", 
                        "rainfall_lag_1",
                        "temperature", 
                        "pressure", 
                        "sunshine_duration", 
                        "wind_speed", 
                        "wind_speed_lag_1")
    
    # Print the schema and stats
    data.describe().show()
    data.show(10)
    data.printSchema()

+-------+------------------+------------------+--------------------+--------------------+------------------+-----------------+------------------+-------------------+-------------------+
|summary|              load|       load_lag_24|            rainfall|      rainfall_lag_1|       temperature|         pressure| sunshine_duration|         wind_speed|   wind_speed_lag_1|
+-------+------------------+------------------+--------------------+--------------------+------------------+-----------------+------------------+-------------------+-------------------+
|  count|             26280|             26280|               26280|               26280|             26280|            26280|             26280|              26280|              26280|
|   mean|27150.157420091324|27146.609741248096|0.057584232418276454|0.057584232418276454|10.736356544901133|959.9136377473403|0.2120966514459675| 1.0579411723621794| 1.0579324867209319|
| stddev| 5199.763342043908| 5200.060296507467| 0.20962309000077545| 0

### Machine Learning

In [58]:
# Keep original timestamp before converting to Unix seconds
unix_time_data = data.withColumn("unix_time", unix_timestamp("time"))  # Store Unix time separately

# Extract day of the year (1-365/366) from the original timestamp column
unix_time_data = unix_time_data.withColumn("day_of_year", dayofyear(col("time")))

# Extract time of day in seconds (seconds since midnight)
unix_time_data = unix_time_data.withColumn(
    "time_of_day", expr("hour(time) * 3600 + minute(time) * 60 + second(time)")
)

# Normalize day of the year to range (0, 2π) for periodic encoding
ml_data = unix_time_data.withColumn("day_sin", sin(2 * 3.1416 * col("day_of_year") / 365))
ml_data = ml_data.withColumn("day_cos", cos(2 * 3.1416 * col("day_of_year") / 365))

# Normalize time of day
ml_data = ml_data.withColumn("time_sin", sin(2 * 3.1416 * col("time_of_day") / 86400))
ml_data = ml_data.withColumn("time_cos", cos(2 * 3.1416 * col("time_of_day") / 86400))

# Add weekend feature
ml_data = ml_data.withColumn("is_weekend", when(col("time").cast("string").like("%Sat%") | col("time").cast("string").like("%Sun%"), lit(1)).otherwise(lit(0)))

# Define new feature columns
feature_columns = ["day_sin", "day_cos", "time_sin", "time_cos", "is_weekend", "rainfall", "temperature", "pressure", "sunshine_duration", "wind_speed"]

feature_columns = [ "load_lag_24",
                    "rainfall", 
                    "rainfall_lag_1",
                    "temperature", 
                    "pressure", 
                    "sunshine_duration", 
                    "wind_speed", 
                    "wind_speed_lag_1"]


assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transform data
df_ml = assembler.transform(ml_data).select("load", "features")

df_ml.cache()

# Show transformed data
df_ml.show(5, truncate=False)



+-------+----------------------------------------------------------------------------------+
|load   |features                                                                          |
+-------+----------------------------------------------------------------------------------+
|21458.0|[22394.0,0.0,0.0,4.45,968.95,0.0,0.6151856390902335,0.6678293725756554]           |
|21151.0|[21804.0,0.0,0.0,3.7,968.6500000000001,0.0,0.30010459245033805,0.6151856390902335]|
|20691.0|[20917.0,0.0,0.0,2.95,968.2,0.0,0.5007752879124893,0.30010459245033805]           |
|20715.0|[20705.0,0.0,0.0,2.65,967.5999999999999,0.0,0.5596157879354227,0.5007752879124893]|
|21302.0|[20965.0,0.0,0.0,2.1,966.9,0.0,0.33647223662121295,0.5596157879354227]            |
+-------+----------------------------------------------------------------------------------+
only showing top 5 rows



In [59]:
scaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = scaler.fit(df_ml).transform(df_ml).select("load", "features_scaled")
scaled_df = scaled_df.withColumnRenamed("features_scaled", "features")
scaled_df.cache()

scaled_df.show(5, truncate=False)

+-------+------------------------------------------------------------------------------------------------------------+
|load   |features                                                                                                    |
+-------+------------------------------------------------------------------------------------------------------------+
|21458.0|[4.306488525727395,0.0,0.0,0.5386610725685395,123.0269329150567,0.0,1.4970013677889624,1.6250823003315231]  |
|21151.0|[4.193028302891851,0.0,0.0,0.4478754985401339,122.98884211586737,0.0,0.7302787269909101,1.496980118810842]  |
|20691.0|[4.02245335771367,0.0,0.0,0.3570899245117284,122.93170591708335,0.0,1.2185936135774293,0.7302683611508879]  |
|20715.0|[3.981684599677847,0.0,0.0,0.3207756949003661,122.85552431870464,0.0,1.3617769121114802,1.2185763164195311] |
|21302.0|[4.031684019910459,0.0,0.0,0.25419960727953544,122.76664578726283,0.0,0.8187762627064287,1.3617575825581258]|
+-------+---------------------------------------

In [60]:
# Randomly splits this :class:`DataFrame` with the provided weights.
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

train_data.take(5)

[Row(load=16055.0, features=DenseVector([3.438, 0.0, 0.0, 1.2892, 121.8525, 0.0, 0.2319, 0.3401])),
 Row(load=16158.0, features=DenseVector([3.1484, 0.0, 0.0, 1.4042, 122.0048, 0.0, 1.7468, 1.6251])),
 Row(load=16168.0, features=DenseVector([3.2938, 0.0, 0.2328, 2.3241, 121.497, 0.0, 3.2804, 3.083])),
 Row(load=16209.0, features=DenseVector([3.449, 0.0, 0.0, 1.386, 121.8525, 0.0, 0.3401, 0.9042])),
 Row(load=16216.0, features=DenseVector([3.1513, 0.0, 0.0, 1.3678, 121.9858, 0.0, 2.0791, 1.7468]))]

In [61]:
# Linear regression.
# The learning objective is to minimize the specified loss function, with regularization.
#More about regParam and elasticNetParam (seen as penalties on the loss function that minimises error in predicitons) can be found here: https://runawayhorse001.github.io/LearningApacheSpark/reg.html
# The maxIter parameter sets an upper limit on the number of iterations the optimization algorithm can perform regarding the loss function
# By default the LinearRegression function expects another column named "features"

lr = LinearRegression(labelCol="load", maxIter=500)

# Fits a model to the input dataset with optional parameters.
linearModel = lr.fit(train_data)

In [62]:
# Define the model
lr = LinearRegression(featuresCol="features", labelCol="load")

# Define hyperparameter grid
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.001, 0.01, 0.1, 1.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])  # 0 = Ridge, 1 = Lasso
             .build())

# Define evaluator
evaluator = RegressionEvaluator(labelCol="load", predictionCol="prediction", metricName="rmse")

# Define cross-validation
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Fit the model with cross-validation
cv_model = cv.fit(train_data)

# Get the best model
best_model = cv_model.bestModel
print(f"Best regParam: {best_model._java_obj.getRegParam()}")
print(f"Best elasticNetParam: {best_model._java_obj.getElasticNetParam()}")

# Make predictions
predictions = best_model.transform(test_data)

# Evaluate performance
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

# Show predictions
predictions.select("load", "prediction").show(10, truncate=False)

Best regParam: 0.01
Best elasticNetParam: 0.5
RMSE: 3090.6906370263464
+-------+------------------+
|load   |prediction        |
+-------+------------------+
|16120.0|18842.321886900776|
|16170.0|20071.18638308376 |
|16555.0|19370.80858682352 |
|16602.0|18888.399310846355|
|16603.0|20371.809304798964|
|16634.0|18917.48732636364 |
|16643.0|19851.170334055467|
|16746.0|18841.606211096874|
|16833.0|20385.929143365403|
|16859.0|19835.989965769855|
+-------+------------------+
only showing top 10 rows



In [64]:
import pandas as pd

# Define feature names
feature_columns = [
    "load_lag_24", "rainfall", "rainfall_lag_1",
    "temperature",
    "pressure", "sunshine_duration",
    "wind_speed", "wind_speed_lag_1"
]

# Get coefficients
coefficients = best_model.coefficients.toArray()

# Create a DataFrame for feature importance
coef_df = pd.DataFrame({"Feature": feature_columns, "Coefficient": coefficients})

# Sort by absolute coefficient values (fixing the issue)
coef_df = coef_df.reindex(coef_df["Coefficient"].abs().sort_values(ascending=False).index)

# Display feature importance
print("\nFeature Importance (Lasso Regression Coefficients):")
print(coef_df)

# Identify removed features (Lasso sets them to exactly 0)
removed_features = coef_df[coef_df["Coefficient"] == 0]["Feature"].tolist()
print(f"\nFeatures removed by Lasso: {removed_features}")




Feature Importance (Lasso Regression Coefficients):
             Feature  Coefficient
0        load_lag_24  3945.184217
3        temperature  -580.703482
5  sunshine_duration   249.906139
6         wind_speed   200.464340
1           rainfall   139.278562
4           pressure   -99.384571
7   wind_speed_lag_1   -89.411928
2     rainfall_lag_1    47.712774

Features removed by Lasso: []


In [65]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Define the Random Forest model
rf = RandomForestRegressor(featuresCol="features", labelCol="load", numTrees=100, maxDepth=10, seed=42)

# Train the Random Forest model
rf_model = rf.fit(train_data)

# Make predictions
rf_predictions = rf_model.transform(test_data)

# Evaluate performance
evaluator = RegressionEvaluator(labelCol="load", predictionCol="prediction", metricName="rmse")
rf_rmse = evaluator.evaluate(rf_predictions)

print(f"Random Forest RMSE: {rf_rmse}")

# Show predictions
rf_predictions.select("load", "prediction").show(10, truncate=False)

# Feature importance
rf_feature_importance = rf_model.featureImportances
print("\nFeature Importances (Random Forest):")
for i, imp in enumerate(rf_feature_importance):
    print(f"{feature_columns[i]}: {imp}")


Random Forest RMSE: 2771.5464666965913
+-------+------------------+
|load   |prediction        |
+-------+------------------+
|16120.0|20128.896720823526|
|16170.0|19377.353153109798|
|16555.0|20179.96231414724 |
|16602.0|19712.619839945473|
|16603.0|19910.2883274851  |
|16634.0|19458.64506745033 |
|16643.0|19915.932504891407|
|16746.0|19349.13494784683 |
|16833.0|19684.32125033349 |
|16859.0|19523.215689663888|
+-------+------------------+
only showing top 10 rows


Feature Importances (Random Forest):
load_lag_24: 0.6959798313303099
rainfall: 0.008815659501222643
rainfall_lag_1: 0.007416601822522744
temperature: 0.1534582691850806
pressure: 0.031865213026882305
sunshine_duration: 0.062015604540655495
wind_speed: 0.022177162942846333
wind_speed_lag_1: 0.018271657650479903
